In [1]:
import numpy as np
import glob
import matplotlib.pyplot as plt
import pickle
import os
import re
import numpy as np

from skimage import measure, segmentation, feature
from vis_utils import load_volume, VolumeVisualizer, ColorMapVisualizer
from scipy.ndimage import zoom
from scipy.signal import fftconvolve
from skimage.morphology import skeletonize_3d, binary_dilation, convex_hull_image
from skimage import filters, morphology
from scipy import signal
from skimage.filters import frangi, sato
from PIL import Image
from tqdm import tqdm

from scipy.ndimage import zoom

## DICE

In [2]:
def dice_coefficient(img1, img2):
    intersection = np.logical_and(img1, img2)
    return 2. * intersection.sum() / (img1.sum() + img2.sum())

## Hausdorff distance

In [3]:
from scipy.spatial.distance import directed_hausdorff

def hausdorff_distance(original, reconstruction):
    original_points_list = np.array(tuple(zip(*np.nonzero(original))))
    reconstr_points_list = np.array(tuple(zip(*np.nonzero(reconstruction))))
    
    original_housdorff = directed_hausdorff(original_points_list, reconstr_points_list)
    reconstr_housdorff = directed_hausdorff(reconstr_points_list, original_points_list)
    
    return original_housdorff, reconstr_housdorff

## Confusion matrix

In [4]:
from sklearn.metrics import confusion_matrix

def conf_matrix(original, reconstruction):
    original_flat = np.ravel(original)
    recons_flat = np.ravel(reconstruction)
    
    conf_mat = confusion_matrix(original_flat, recons_flat)
    
    TN, FP, FN, TP = conf_mat.ravel()
    precision = TP / (TP + FP)
    accuracy = (TP + TN) / (TP + FP + TN + FN)
    recall = TP / (TP + FN)
    
    return precision, accuracy, recall

# Results 

In [9]:
def get_volume(filename):
    try:
        tokens = re.split(r'x|_|\.', filename)
        shape_z, shape_y, shape_x = int(tokens[-4]), int(tokens[-3]), int(tokens[-2])
        volume = np.fromfile(filename, dtype=np.uint8)
        return volume.reshape(shape_x, shape_y, shape_z)
    except:
        print("Invalid filename, correct format: <filename>_<shape x>x<shape y>x<shape z>.raw")

In [ ]:
files = os.listdir("reconstructed_M_full")
# results = open("model_evaluation_results.txt", "w")

for i, file in enumerate(files):
    if i != 11 and i != 14:
        continue
    file_path = os.path.join("reconstructed_M_full", file)
    reconstruction = np.load(file_path)

    prep_no = re.split(r"[_\.]", file_path)[-2]
    
    org_path = "../data/" + prep_no + "/"
    org_files = os.listdir(org_path)
    org_files = [file for file in org_files if file.endswith(".npy")]
    org_file = org_files[0]
    
    original = np.load(org_path + org_file)
#     original = get_volume(org_path + org_file)
    original = original > 32
    
    x,y,z = original.shape[0] - reconstruction.shape[0], original.shape[1] - reconstruction.shape[1], original.shape[2] - reconstruction.shape[2]
    
    original = original[x//2 : original.shape[0] - (x - x//2), y//2 : original.shape[1] - (y - y//2), z//2 : original.shape[2] - (z - z//2),]
    print(original.shape, reconstruction.shape)
#     results.write(str(prep_no) + '\n')
    
    dice_score = dice_coefficient(original, reconstruction)
    precision, accuracy, recall = conf_matrix(original, reconstruction)

#     results.write(f"Dice coefficient: {dice_score}\n")
    print(prep_no, i)
    print(f"Dice coefficient: {dice_score}")
    print(f"Precision: {precision}\n")
    print(f"Accuracy: {accuracy}\n")
    print(f"Recall: {recall}\n")
    
#     original_housdorff, reconstr_housdorff = hausdorff_distance(original, reconstruction)
#     results.write("Directed housdorff (orginal): " + str(original_housdorff) + '\n')
#     results.write("Directed housdorff (reconstruction): " + str(reconstr_housdorff) + '\n')
    
#     results.write(f"Precision: {precision}\n")
#     results.write(f"Accuracy: {accuracy}\n")
#     results.write(f"Recall: {recall}\n")
    
#     results.flush()
    
# results.close()